In [1]:
!pwd

/home/sagemaker-user/hjoshi/Lesion_Tracker_Liver


In [2]:
!pip install nibabel totalsegmentator==2.4.0 nbformat -qU

In [3]:
!pip install opencv-python-headless

  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (54.0 MB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python-headless]v-python-headless]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.3.1 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
amazon-sagemaker-jupyter-ai-q-developer 1.2.7 requires numpy<=

In [4]:
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt
from totalsegmentator.python_api import totalsegmentator
import os
import glob
from tqdm import tqdm
import numpy as np
# import cv2
import tarfile
import shutil

import SimpleITK as sitk
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from skimage.measure import centroid
from concurrent.futures import ThreadPoolExecutor
from enum import Enum
import sys
import matplotlib.pyplot as plt
import nibabel as nib 

from enum import Enum
import pandas as pd
from pydantic import BaseModel

/opt/conda/lib/python3.12/site-packages/totalsegmentator/config.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [13]:
seg_path = "data/labels_liver_cropped/"

In [14]:
def read_nifti(path):
    img = nib.load(path)
    img_data = img.get_fdata()
    img_affine = img.affine
    img_header = img.header
    return img, img_data

In [15]:
def create_tumor_seg(path, save_path):
    seg, seg_data = read_nifti(path)
    seg_data[seg_data == 1] = 0 
    tumor_img = nib.Nifti1Image(seg_data, affine=seg.affine)
    output_path = f'{save_path}/labels_2/{path.split('/')[-1]}.gz'
    nib.save(tumor_img, output_path)
    return output_path

In [8]:
images_path = "data/images/"

In [16]:
path_list = os.listdir(seg_path)

In [17]:
for idx in range(len(path_list)):
    path = os.path.join(seg_path, path_list[idx])
    seg, seg_data = read_nifti(path)
    seg_data[np.where(seg_data == 1)] = 0
    seg_data[np.where(seg_data == 2)] = 1
    tumor_img =  nib.Nifti1Image(seg_data, affine=seg.affine)
    output_path = f'data/labels_tumor_cropped/{path_list[idx]}'
    nib.save(tumor_img, output_path)

In [12]:
def create_and_crop_liver_data(image_folder, labels_folder, output_image_folder, output_labels_folder):
    """
    Filters a CT scan to show only liver/tumor HU values, then crops both the
    CT and label files based on the liver's z-range and the bounding box of 
    the largest 2D segmented slice.
    """

    for folder in [output_image_folder, output_labels_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    for filename in os.listdir(image_folder):
        if filename.endswith(".nii"):
            image_path = os.path.join(image_folder, filename)
            label_path = os.path.join(labels_folder, filename)

            if not os.path.exists(label_path):
                print(f"Skipping {filename} as no corresponding label file was found.")
                continue

            # Load the data
            img = nib.load(image_path)
            label = nib.load(label_path)
            
            img_data = img.get_fdata()
            label_data = label.get_fdata()

            if img_data.shape != label_data.shape:
                print(f"Shapes of {filename} image and label do not match. Skipping.")
                continue

            # Step 1: Find the largest segmented slice and its x,y bounding box
            slice_voxel_counts = np.sum(label_data > 0, axis=(1, 2))
            if np.sum(slice_voxel_counts) == 0:
                print(f"No liver segmentation found in {filename}. Skipping.")
                continue
            
            largest_slice_index = np.argmax(slice_voxel_counts)
            largest_slice = label_data[largest_slice_index, :, :]
            
            coords_xy = np.argwhere(largest_slice > 0)
            if coords_xy.size == 0:
                print(f"Largest slice found but is empty. Skipping {filename}.")
                continue
            
            min_y, min_x = coords_xy.min(axis=0)
            max_y, max_x = coords_xy.max(axis=0)
            
            buffer = 10
            min_y = max(0, min_y - buffer)
            min_x = max(0, min_x - buffer)
            max_y = min(img_data.shape[1], max_y + buffer)
            max_x = min(img_data.shape[2], max_x + buffer)

            # Step 2: Find the z-range (slices) where the liver exists
            z_coords = np.argwhere(slice_voxel_counts > 0)
            min_z = z_coords.min()
            max_z = z_coords.max()
            
            # Step 3: Apply the cropping to BOTH the image data and the label data
            
            # Create a mask to get only liver/tumor HU values, then crop
            mask = np.where(label_data > 0, 1, 0)
            liver_only_data = img_data * mask
            cropped_img_data = liver_only_data[min_z:max_z, min_y:max_y, min_x:max_x]
            
            # Crop the original label data using the same coordinates
            cropped_label_data = label_data[min_z:max_z, min_y:max_y, min_x:max_x]

            # Step 4: Save the new images and labels
            output_filename = filename.replace(".nii", ".nii.gz")
            
            # Save the cropped image
            cropped_img = nib.Nifti1Image(cropped_img_data, img.affine)
            output_img_path = os.path.join(output_image_folder, output_filename)
            nib.save(cropped_img, output_img_path)
            
            # Save the cropped label
            cropped_label = nib.Nifti1Image(cropped_label_data, label.affine)
            output_label_path = os.path.join(output_labels_folder, output_filename)
            nib.save(cropped_label, output_label_path)
            
            print(f"Created and cropped {output_filename} for both image and label.")

# Define your folder paths
images_folder = 'data/images'
labels_folder = 'data/labels_liver_and_tumor'
output_folder = 'data/images_liver_2'
output_labels_folder = 'data/labels_liver_cropped'

# Run the function
create_and_crop_liver_data(images_folder, labels_folder, output_folder, output_labels_folder)

Created and cropped liver_0.nii.gz for both image and label.
Created and cropped liver_1.nii.gz for both image and label.
Created and cropped liver_10.nii.gz for both image and label.
Created and cropped liver_101.nii.gz for both image and label.
Created and cropped liver_102.nii.gz for both image and label.
Created and cropped liver_104.nii.gz for both image and label.
Created and cropped liver_105.nii.gz for both image and label.
Created and cropped liver_106.nii.gz for both image and label.
Created and cropped liver_107.nii.gz for both image and label.
Created and cropped liver_108.nii.gz for both image and label.
Created and cropped liver_109.nii.gz for both image and label.
Created and cropped liver_11.nii.gz for both image and label.
Created and cropped liver_110.nii.gz for both image and label.
Created and cropped liver_111.nii.gz for both image and label.
Created and cropped liver_112.nii.gz for both image and label.
Created and cropped liver_113.nii.gz for both image and label